In [2]:
#!pip install spacy-langdetect

     -------------------------------------- 998.1/998.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993439 sha256=3cc6d52711d958e89c4c8aff5fa1ab9534bd210a987bcc73954d29297a8d83f1
  Stored in directory: c:\users\wei.liu\appdata\local\pip\cache\wheels\87\8c\9a\41c0647bd03b3e11ca6968d3638a4e6e764220adf2886270cb
Successfully built langdetect


In [3]:
#!python -m spacy download en_core_web_lg

     -------------------------------------- 587.7/587.7 MB 1.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [1]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance
from hdbscan import HDBSCAN
import hdbscan
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import v_measure_score
from umap import UMAP

import time
from unidecode import unidecode
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

In [2]:
# load processed data from clean folder
df_1819 = pd.read_csv('data/clean/clean_uncat_app_pre.csv')
df_labeled_1819 = pd.read_csv('data/clean/cleaner_labeled_data_1819.csv',index_col=0)


In [4]:
df_1819.shape

(19135, 7)

## Step 2.3 Shuffle Data

In [5]:
# shuffle first because some requests may be seasonal, such as drainage, shuffle to avoid the data being time biased
df_1819s = df_1819.sample(frac=1, random_state=123, ignore_index=True)
#df_1819s_1 = df_1819s.iloc[0:int(.7*df_1819s.shape[0]),:]
#df_18s_2 = df_18s.iloc[int(.5*df_18s.shape[0]):, :]

## Step 3. Train-Test split

In [6]:
# split labeled data into train, valid, and test sets
# train set merge with other_issues to perform semi-supervised learning
# valid set for tuning hyper-parameters, test set for testing

# shuffle 
df_labeled_1819s = df_labeled_1819.sample(frac=1, random_state=123, ignore_index=True)
train_set = df_labeled_1819s.iloc[:int(0.7*df_labeled_1819s.shape[0]),:]
val_set = df_labeled_1819s.iloc[int(0.7*df_labeled_1819s.shape[0]):int(0.85*df_labeled_1819s.shape[0]),:]
tst_set = df_labeled_1819s.iloc[int(0.85*df_labeled_1819s.shape[0]):, :]

In [7]:
df_cmb_1 = pd.concat([df_1819s, train_set], axis=0, ignore_index=True)

print(df_cmb_1.shape)

df_cmb_1.head()

(23171, 7)


,Incident_ID,Date_Created,Description,Service,len,lang,semi_label
0,413481.0,2019-10-12 10:47:00,Illegal red zone parking. Blocking view of inc...,Other Issues,25.0,en,-1.0
1,299718.0,2019-03-17 12:35:00,"I’m not sure what’s going on here, most days d...",Other Issues,39.0,en,-1.0
2,380373.0,2019-08-14 17:42:15,At 1029 Branham lane the The renters living th...,Other Issues,34.0,en,-1.0
3,325921.0,2019-05-07 23:31:00,Abandoned vehicle with a long trailer is creat...,Other Issues,19.0,en,-1.0
4,362320.0,2019-07-15 14:17:07,there is a rv parked on corner of coyote and h...,Other Issues,27.0,en,-1.0


In [8]:
# The original data, used for prediction after model tuning
doc_1819 = df_1819['Description'].to_list()

doc_cmb_1 = df_cmb_1['Description'].to_list()

doc_trn = train_set['Description'].to_list()
doc_val = val_set['Description'].to_list()
doc_tst = tst_set['Description'].to_list()


y_cmb_1 = df_cmb_1['semi_label'].to_list()

y_trn = train_set['semi_label'].to_list()
y_val = val_set['semi_label'].to_list()
y_tst = tst_set['semi_label'].to_list()



# Semi-Supervised Model 

In [9]:

# Sentence Transformers convert docs (service requests) into numeric representations
#sentence_model = SentenceTransformer("all-mpnet-base-v2")
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# regenerate key words for topic representation
representation_model = KeyBERTInspired()
#representation_model = MaximalMarginalRelevance(diversity=0.5)

vectorizer_model = CountVectorizer(stop_words="english")


In [10]:
# pre-calculate embeddings to save time
st = time.time()
d1819_embeddings = sentence_model.encode(doc_1819, show_progress_bar=False)

cmb1_embeddings = sentence_model.encode(doc_cmb_1, show_progress_bar=False)

trn_embeddings = sentence_model.encode(doc_trn, show_progress_bar=False)
val_embeddings = sentence_model.encode(doc_val, show_progress_bar=False)
tst_embeddings = sentence_model.encode(doc_tst, show_progress_bar=False)
elapsed = time.time()-st
print("Run time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))

Run time: 00:14:59.50


## Use the default setting to set up a baseline

In [11]:
st = time.time()
topic_model_base = BERTopic()
                        
base_topics, base_probs = topic_model_base.fit_transform(doc_cmb_1, cmb1_embeddings, y=y_cmb_1)



y_trn_base, tr_probs_base = topic_model_base.transform(doc_trn, trn_embeddings)
y_val_base, val_probs_base = topic_model_base.transform(doc_val, val_embeddings)
print(v_measure_score(y_trn_base, y_trn))
print(v_measure_score(y_val_base, y_val))

elapsed = time.time()-st
print("Run time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))

C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


0.26757974249588506
0.2645086250643478
Run time: 00:01:50.26


## Random Search
There are 4 hyper-parameters that are important for the performance of Bertopic model:  
2 hyper-parameters for umap, which is for dimension reduction;  
and 2 hyper-parameters for hdbscan, which is for clustering documents.  
Unfortunately, due to the limitation of computing power and time, we can not use grid search for hyper-parameters tuning, which will lead to thounsands of times of model training and testing.  
Instead we'll implement a Random search.

In [12]:
# Generate n sets of random parameters in certain range

# number of parameter sets
n = 40

np.random.seed(1)
# n_neighbors
nnbs = np.random.randint(low=5, high=30, size=n)
# n_component
ncpts = np.random.randint(low=5,high=30, size=n)
# min_cluster_size
mc_sizes = np.random.randint(low=10, high=60, size=n)
# min_sample_size
ms_sizes = np.random.randint(low=10, high=60, size=n)

In [13]:

# Sentence Transformers convert docs (service requests) into numeric representations
#sentence_model = SentenceTransformer("all-mpnet-base-v2")
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# regenerate key words for topic representation
representation_model = KeyBERTInspired()
#representation_model = MaximalMarginalRelevance(diversity=0.5)

vectorizer_model = CountVectorizer(stop_words="english")
metric='euclidean'

In [14]:
st = time.time()
tr_score, val_score, cnt_topics = [],[],[]
for nnb,ncpt,mcs,mss in zip(nnbs, ncpts,mc_sizes,ms_sizes):
    # UMAP model handles demension reduction which can be helpful with large datasets
    umap_model = UMAP(n_neighbors=nnb, n_components=ncpt, metric=metric, random_state=123)
    # HDB Scan works on clustering groups of similar embeddings
    hdbscan_model = HDBSCAN(min_cluster_size=mcs, min_samples=mss, prediction_data=True)
    
    topic_model_cmb1 = BERTopic(nr_topics="auto", umap_model=umap_model,embedding_model=sentence_model, hdbscan_model=hdbscan_model, \
                       representation_model=representation_model, vectorizer_model=vectorizer_model,\
                            min_topic_size=mcs, calculate_probabilities=True)
                        
    cmb1_topics, cmb1_probs = topic_model_cmb1.fit_transform(doc_cmb_1, cmb1_embeddings, y=y_cmb_1)

    
    cnt = max(cmb1_topics)
    cnt_topics.append(cnt)
        
    y_trn_pred, tr_probs = topic_model_cmb1.transform(doc_trn, trn_embeddings)
    y_val_pred, val_probs = topic_model_cmb1.transform(doc_val, val_embeddings)
        
    tr_score.append(v_measure_score(y_trn_pred, y_trn))
    val_score.append(v_measure_score(y_val_pred, y_val))
    
elapsed = time.time()-st
print("Run time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))

C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-pa

C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-pa

Run time: 01:49:34.24


In [15]:
d_scores = {'params':zip(nnbs, ncpts,mc_sizes,ms_sizes),'training_score':tr_score, 'val_score':val_score, 'topic_count':\
           cnt_topics}
gd_scores = pd.DataFrame(data=d_scores)
gd_scores.sort_values(by='val_score',ascending=False, inplace=True,ignore_index=True)

gd_scores

,params,training_score,val_score,topic_count
0,"(20, 12, 57, 23)",0.602032,0.589308,25
1,"(21, 11, 35, 33)",0.597072,0.581978,37
2,"(22, 24, 46, 25)",0.589794,0.576919,25
3,"(27, 13, 17, 51)",0.597789,0.576715,23
4,"(25, 16, 33, 42)",0.594284,0.575079,22
5,"(17, 18, 42, 55)",0.598554,0.572261,22
6,"(17, 22, 38, 17)",0.571514,0.570874,46
7,"(22, 7, 37, 44)",0.593233,0.570356,30
8,"(23, 23, 16, 43)",0.582585,0.564351,28
9,"(27, 28, 53, 17)",0.517959,0.530448,26


### The optimal model: Although the validation score is a very critical metric for the evaluation of the model performance, there are other factor to consider here, such as number of topics, number of outliers, and the saliency of the topics, after experimenting on the top 3 sets of parameter, the following set of hyper parameters turn out to be the optimal. 

In [113]:
# hyper-params that gain best val_score
n_nbors, n_components, min_cluster_size, min_samples = (17, 22, 38, 17)


In [127]:
#UMAP model handles demension reduction which can be helpful with large datasets
umap_model = UMAP(n_neighbors=n_nbors, n_components=n_components, metric=metric, random_state=123)
# HDB Scan works on clustering groups of similar embeddings
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
st = time.time()
topic_model_op_1819 = BERTopic(nr_topics="auto", umap_model=umap_model,embedding_model=sentence_model, hdbscan_model=hdbscan_model, \
                       representation_model=representation_model, vectorizer_model=vectorizer_model,\
                            min_topic_size=min_cluster_size, calculate_probabilities=True)
                        
cmb1_topics, cmb1_probs = topic_model_op_1819.fit_transform(doc_cmb_1, cmb1_embeddings, y=y_cmb_1)



y_trn_pred_1, tr_probs_1 = topic_model_op_1819.transform(doc_trn, trn_embeddings)
y_val_pred_1, val_probs_1 = topic_model_op_1819.transform(doc_val, val_embeddings)
print(v_measure_score(y_trn_pred_1, y_trn))
print(v_measure_score(y_val_pred_1, y_val))

elapsed = time.time()-st
print("Run time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))

C:\Users\Wei.Liu\AppData\Local\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


0.5715141484133803
0.5708737243642917
Run time: 00:03:54.09


## Examine the Optimal model

In [128]:
# Model was assessed on how well-defined topics were and what topics were created. Hyperparameters can be adjusted to get different topics.
# The key to find a balance between topics that are specific but not too granular
pd.set_option('display.max_rows', None)
topic_model_op_1819.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6135,-1_husted_abandoned_rv_government,"[husted, abandoned, rv, government, public, wo...",[The government workers aren’t able to do anyt...
1,0,4122,0_parked_parking_sidewalk_park,"[parked, parking, sidewalk, park, driveway, ve...","[Always parking on the sidewalk, Car parked in..."
2,1,3360,1_homeless_shelters_shelter_neighborhood,"[homeless, shelters, shelter, neighborhood, st...","[Homeless area. Please assist. Thank you., H..."
3,2,1689,2_potholes_pothole_roadway_pavement,"[potholes, pothole, roadway, pavement, road, p...","[large pothole at corner & broken down road, p..."
4,3,1590,3_laura_government_rv_mile,"[laura, government, rv, mile, enforce, commitm...",[Hey government workers! Time to get to work! ...
5,4,755,4_tree_trees_pine_street,"[tree, trees, pine, street, branches, branch, ...","[Broken tree, Tree down blocking street, Tree ..."
6,5,664,5_leaking_leak_leaks_drainage,"[leaking, leak, leaks, drainage, drain, puddle...","[Water pipe leaking water, water drain seems t..."
7,6,512,6_sign_broken_signs_signage,"[sign, broken, signs, signage, damaged, repair...","[Broken sign, Broken street sign., Street sign..."
8,7,410,7_panhandling_panhandlers_panhandler_area,"[panhandling, panhandlers, panhandler, area, a...",[Panhandling area. Please assist. Thank you....
9,8,396,8_weeds_weed_overgrown_lawn,"[weeds, weed, overgrown, lawn, grass, landscap...",[Overgrown front yard with nothing but tall we...


In [117]:
df_cmb_1['Description'][np.array(cmb1_topics)==21][:80].values

array(['=&gt;You have to love the government workers e-mail to me -  Hello Thomas,   Thank you for contacting the City of San Jose\'s Customer Contact Center via My San Jose.  Your request, incident #191031-000521, will be forwarded to the Department of Transportation. If needed, please contact them directly to follow up on your request about:   "The government workers aren’t able to do anything about the RV at Husted and Booksin. They are most productive when they close my requests. I’ll just wait until the government workers leave for the day and then submit them again. What happened to Laura Wells? What does she do for all our tax money? Why hasn’t Jim Ortbal returned my phone calls?"   You can find answers to inquiries, report issues and more by going to My San Jose at: https://sanjose.custhelp.com/  Search FAQs to get information about San Jose services and more.   If you have further questions, we can be reached at the number below.   =&gt;Isn’t that what I’ve been doing?',
     

In [23]:
topic_model_op_18.save('model_cmb_op_18')

## topic merging

In [129]:
topics_to_merge_1819 = [[0,3,15,24,42],[1,7],[2,38],[4,8],[5,27],[9,11,17,44],[13,26],[16,21],[18,20,23,35,39,45,46],[19,34],
                        [25,31,37],[30,33]]
topic_model_op_1819.merge_topics(doc_cmb_1, topics_to_merge_1819)

cmb1_mg_topics = topic_model_op_1819.topics_[:]

topic_model_op_1819.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6135,-1_husted_abandoned_rv_government,"[husted, abandoned, rv, government, tax, publi...",[The government workers aren’t able to do anyt...
1,0,5997,0_laura_government_rv_mile,"[laura, government, rv, mile, enforce, commitm...",[Hey government workers! Time to get to work! ...
2,1,3770,1_homeless_shelters_shelter_neighborhood,"[homeless, shelters, shelter, neighborhood, ar...","[Homeless area. Please assist. Thank you., H..."
3,2,1731,2_potholes_pothole_manhole_paved,"[potholes, pothole, manhole, paved, roadway, p...","[Pothole near middle of the road., pothole in ..."
4,3,1151,3_tree_trees_pine_leaves,"[tree, trees, pine, leaves, road, street, bran...","[Tree down blocking street, Tree down, Tree down]"
5,4,758,4_cart_carts_shopping_grocery,"[cart, carts, shopping, grocery, bike, parking...","[Shopping cart, Shopping cart, Shopping cart]"
6,5,731,5_hydrant_leaking_hydrants_leak,"[hydrant, leaking, hydrants, leak, leaks, pudd...","[Blocking water hydrant, Fire hydrant leaking ..."
7,6,512,6_sign_street_signage_broken,"[sign, street, signage, broken, signs, road, a...","[Broken street sign, Broken street sign., Stre..."
8,7,472,7_dead_cat_deceased_cats,"[dead, cat, deceased, cats, pet, dog, animal, ...","[Dead cat, Dead dog on road, Dead Dog]"
9,8,250,8_streetlights_lights_streetlight_lighting,"[streetlights, lights, streetlight, lighting, ...","[Street light out., The street light is ?????,..."


In [130]:
y_trn_pred, trn_probs = topic_model_op_1819.transform(doc_trn, trn_embeddings)
y_val_pred, val_probs = topic_model_op_1819.transform(doc_val, val_embeddings)


print(v_measure_score(y_trn_pred, y_trn))
print(v_measure_score(y_val_pred, y_val))

0.5938433108631521
0.5851224350136733


In [131]:
# Predict on 2018 other issues data
st = time.time()
df_1819_topics, df_1819_probs = topic_model_op_1819.transform(doc_1819, d1819_embeddings)
elapsed = time.time()-st
print("Run time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))

Run time: 00:02:50.01


## reduce outliers

In [132]:
# valiation score after topic merging and outlier reduction 
dist_val_pred = topic_model_op_1819.reduce_outliers(doc_val, y_val_pred, strategy="distributions", distributions_params={'window':3,'stride':1})
v_measure_score(dist_val_pred, y_val)

0.5771267216417743

In [92]:
print(f'number of potholes in val_set:{val_set[val_set.semi_label==0].shape[0]}')
print(f'number of parking in val_set: {val_set[val_set.semi_label==1].shape[0]}')
print(f'number of trash in val_set: {val_set[val_set.semi_label==2].shape[0]}')

number of potholes in val_set:318
number of parking in val_set: 304
number of trash in val_set: 243


In [133]:
print(100*sum((np.array(dist_val_pred)==2)&(val_set.semi_label==0))/val_set[val_set.semi_label==0].shape[0])
print(100*sum((np.array(dist_val_pred)==0)&(val_set.semi_label==1))/val_set[val_set.semi_label==1].shape[0])
print(100*(sum((np.array(dist_val_pred)==1)&(val_set.semi_label==2))+sum((np.array(dist_val_pred)==4)&(val_set.semi_label==2)))/val_set[val_set.semi_label==2].shape[0])

92.45283018867924
77.96052631578948
83.95061728395062


In [94]:
print('Example of mis-clustered pothole reports:')
val_set[(np.array(dist_val_pred)!=2)&(val_set.semi_label==0)]['Description'][:10].values

Example of mis-clustered pothole reports:


array(['It’s easy to tip over here',
       'Shopping cart with trash on Lone Bluff Way ; corner of Dakota Drive on street by golf course',
       'c/s Cinnabar St - in middle of street - RP got a popped tire-',
       'btwn market place driveway & autumn',
       'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi at purus sem. Quisque quis nisi id leo ornare placerat. Sed vitae urna ullamcorper, lacinia risus id, rhoncus lacus. Fusce cursus turpis massa, sed volutpat lectus rutrum non. Donec consectetur gravida gravida. Curabitur rutrum tempus nisl. Praesent nec tortor sit amet erat tincidunt hendrerit. Curabitur enim nisi, ullamcorper a orci nec, commodo commodo lectus.  Curabitur aliquet lorem at purus vulputate ullamcorper. Phasellus vehicula quis purus lacinia consequat. Aenean pharetra posuere orci vitae auctor. Donec rhoncus ultrices diam a interdum. Vestibulum eleifend lectus ultrices, aliquam odio ut, consectetur lorem. Cras laoreet finibus nibh, a porttitor tellu

In [95]:
print('Example of mis-clustered parking related reports:')
val_set[(np.array(dist_val_pred)!=0)&(val_set.semi_label==1)]['Description'][:10].values

Example of mis-clustered parking related reports:


array(["The car was seen being dumped there by a towe company. It has been sitting and not moved in over a week. It is a white chevy El Camino. There is junk inside the car and looks like it doesn't run.",
       'Black Jetta with words Giovanny on back of the trunk tinted windows.',
       'Window down on driver side, black cover on tire attached to back of car with the letters CR-V',
       'vehicle being store and repair on the street for some week',
       'Truck sitting there for 7 weeks already broken down. Danger when someone keep opening the hood hard to see traffic coming on Hillmont to make a turn left turn from sundown lane. Also saw two home guys at the truck with the hood open up never seen them around here or that truck. Please remove it children always walking by there to school.  Truck on Hillmont ave and cross street Ocala.',
       'Parked in front. Of my residence for at least 10 days',
       'This car has been parked in front of my house on Corner of Skyfarm and Sp

In [97]:
print('Example of mis-clustered trash related reports:')
val_set[(np.array(dist_val_pred)!=0)&(np.array(dist_val_pred)!=64)&(val_set.semi_label==2)]['Description'][:10].values

Example of mis-clustered trash related reports:


array(['Trash along 7th street side walk between Jackson and Empire Street.  Mattress, drawers, parts of cabinet, other trash.  See attached pictures.',
       'Furniture and trash from a homeless encampment',
       'Some persons dumped couch, chairs boxes on sidewalk.',
       'Old tire on sidewalk across the street from 6918 Port Rowan Drive',
       'Two big mattresses left on the side of the road 2 days ago. One is on the bushes and one is leaned up against the fence facing target',
       'cs Northlake Dr 3 bags of garb',
       'Mattress and trash in the median area on Monterey near Fehern Dr.',
       'Hi, this has been sitting in front of this house for more than one month.  No action.  Owner apparently thinks it has some value to someone as scap metal  thanks,',
       'Again with the mattresses left for 2 weeks now',
       'Garbage and recycling mixed together. Lots on ground'],
      dtype=object)

In [134]:
# test set score
y_tst_pred, tst_probs = topic_model_op_1819.transform(doc_tst, tst_embeddings)
dist_tst_pred = topic_model_op_1819.reduce_outliers(doc_tst, y_tst_pred, strategy='distributions', distributions_params={'window':3,'stride':1})
v_measure_score(dist_tst_pred, y_tst)

0.6015412065094434

In [135]:
d1819_new_topics = topic_model_op_1819.reduce_outliers(doc_1819, df_1819_topics, strategy="distributions", distributions_params={'window':3,'stride':1})

In [136]:
topic_model_op_1819.update_topics(docs=doc_1819, topics=d1819_new_topics)
topic_model_op_1819.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7,-1_vehichle_plpe_oncw_absndoned,"[vehichle, plpe, oncw, absndoned, singh, back,...",[The government workers aren’t able to do anyt...
1,0,5987,0_the_do_government_workers,"[the, do, government, workers, to, wells, laur...",[Hey government workers! Time to get to work! ...
2,1,3653,1_homeless_thank_area_assist,"[homeless, thank, area, assist, please, you, t...","[Homeless area. Please assist. Thank you., H..."
3,2,564,2_road_lane_the_cover,"[road, lane, the, cover, is, of, hole, on, and...","[Pothole near middle of the road., pothole in ..."
4,3,1479,3_tree_weeds_trees_sidewalk,"[tree, weeds, trees, sidewalk, overgrown, the,...","[Tree down blocking street, Tree down, Tree down]"
5,4,912,4_shopping_cart_remove_street,"[shopping, cart, remove, street, abandoned, ca...","[Shopping cart, Shopping cart, Shopping cart]"
6,5,893,5_water_drain_hydrant_fire,"[water, drain, hydrant, fire, storm, leaking, ...","[Blocking water hydrant, Fire hydrant leaking ..."
7,6,912,6_sign_stop_signs_street,"[sign, stop, signs, street, on, is, the, down,...","[Broken street sign, Broken street sign., Stre..."
8,7,540,7_dead_dog_cat_animal,"[dead, dog, cat, animal, in, dogs, on, barking...","[Dead cat, Dead dog on road, Dead Dog]"
9,8,581,8_light_lights_traffic_the,"[light, lights, traffic, the, on, is, signal, ...","[Street light out., The street light is ?????,..."


In [112]:
df_1819['Description'][np.array(d1819_new_topics)==8]

7        Park view 3 park off monet place in south sj (...
18       Train barriers are broken. Traffic is at a sta...
23       Large debis piles on entrance to Xander Crossi...
84              Bicycle pump hose cut and stolen by vandel
87         Portable Basketball Hoop left out on the street
100                        Broken lock on electrical panel
101      Beige walls are crumbling on Capitol. Is this ...
163      Basketball hoop is out in the street.  Garbage...
182      I live at the end of a dead end street...there...
200                                Fix slide in playground
214      There is a broken cable railing on the old tre...
234      condoms found on ground this morning.  camera ...
250      I live on the property and the neighbor to the...
273      Cars parked on lawn, garbage all over property...
274      Someone took our mail from the mail box and pu...
277      Vandals have cut through fence with wire cutte...
278      Metal fence bars have been removed, allowing h.

In [137]:
df_1819['final_label'] = d1819_new_topics
df_1819.head()

,Incident_ID,Date_Created,Description,Service,len,lang,semi_label,final_label
0,99590,2018-01-01 03:26:17,Signal doesn’t cycle for left turns onto San S...,Other Issues,9,en,-1,0
1,99611,2018-01-01 09:46:07,Vehicles parked in front yard.,Other Issues,5,en,-1,0
2,99612,2018-01-01 09:47:31,Vehicle parked in front yard.,Other Issues,5,en,-1,0
3,99629,2018-01-01 11:42:09,Parked illegally in red zone,Other Issues,5,en,-1,0
4,99634,2018-01-01 11:57:35,Abandoned Safeway shopping cart. On sidewalk,Other Issues,6,en,-1,4


In [138]:
final_label_dict = {-1: 'Other',
              0:'Parking Violation',
              1:'Homeless/Panhandler',
              2:'Pothole',
              3:'Plant',
              4:'Trash',
              5:'Sewage/Drainage',
              6:'Street/Sidewalk Infra',
              7:'Animal',
              8:'Traffic Light',
              9:'Request Follow-up',
              10:'Fence',
              11:'Speeding',
              12:'Noise',
              13:'Parks/Play Ground',
              14:'Water Leakage',
              15:'Alcohol/Smoking/Drug',
              16:'Account',
              17:'Fire/Firework',
              18:'Neighboorhood Complaint',
              19:'Utility Cable/Box',
              20:'Graffiti',
              21:'Utility Bill/Service',
              22:'Suspecious People/Behavior',
              23:'Location Only'}

In [139]:
df_1819['final_label_verb'] = df_1819['final_label'].apply(lambda x: final_label_dict[x])
df_1819.head()

,Incident_ID,Date_Created,Description,Service,len,lang,semi_label,final_label,final_label_verb
0,99590,2018-01-01 03:26:17,Signal doesn’t cycle for left turns onto San S...,Other Issues,9,en,-1,0,Parking Violation
1,99611,2018-01-01 09:46:07,Vehicles parked in front yard.,Other Issues,5,en,-1,0,Parking Violation
2,99612,2018-01-01 09:47:31,Vehicle parked in front yard.,Other Issues,5,en,-1,0,Parking Violation
3,99629,2018-01-01 11:42:09,Parked illegally in red zone,Other Issues,5,en,-1,0,Parking Violation
4,99634,2018-01-01 11:57:35,Abandoned Safeway shopping cart. On sidewalk,Other Issues,6,en,-1,4,Trash


In [140]:
df_1819.to_csv('data/result/Result_uncat_pre_app.csv', index=False)